# Implementing Byte Pair Encoding (BPE) for Tokenization: A Step-by-Step Guide

In the book “Build a Large Language Model (From Scratch)”, Sebastian Raschka introduces various ways to process text data for large language models (LLMs). In Chapter 2, he covers basic tokenization methods, but also mentions more advanced techniques like Byte Pair Encoding (BPE), noting that implementing it is beyond the book’s scope. However, for those curious about diving deeper, I took on the challenge to implement a simple BPE tokenizer myself.

In this article, I will walk you through:

## Why BPE is crucial for tokenization.

Why avoiding UNK tokens is essential for effective language processing.
A simple BPE implementation that breaks down text into subword tokens.
The code I wrote to process text, and the statistics it produces.

## Why Byte Pair Encoding?

Tokenization is the first step for any language model, transforming raw text into “tokens” that the model can understand. For example, simple tokenizers may treat each word as a token or break it down further into characters. However, this approach doesn’t handle out-of-vocabulary words well and can inflate the number of tokens, especially for rare words.

Here’s where BPE comes into play. BPE compresses the text by merging the most frequent pairs of characters or subwords iteratively. It effectively balances tokenization by handling common words as whole tokens while splitting rare words into subword tokens. This allows LLMs to process unknown words without overwhelming the model with too many individual tokens.

## The Importance of Avoiding UNK Tokens

When building language models, encountering unknown tokens (UNK) can be detrimental to performance. An UNK token represents any word that is not present in the model’s vocabulary, and while it’s a common fallback in traditional tokenization approaches, it introduces several significant drawbacks:

- Loss of Information: When a model encounters an UNK token, it loses all information about that word. This means that any context, meaning, or nuance associated with that word is entirely discarded.
- Inefficiency in Language Representation: UNK tokens can lead to inefficient representations of language. When a significant portion of the text consists of UNK tokens, the model is forced to work with a distorted understanding of the input.
- Inability to Generalize: The presence of UNK tokens hampers a model’s ability to generalize from training data to unseen text. If the model has not learned to associate certain words or phrases with specific contexts, it may struggle with understanding similar phrases or variations.
- Impact on Downstream Tasks: For tasks such as translation or summarization, encountering UNK tokens can severely degrade performance, leading to misinterpretations of user intent.

## How BPE Mitigates the UNK Token Problem

Byte Pair Encoding (BPE) addresses the issue of UNK tokens effectively by breaking down words into smaller, more manageable subword units. Here’s how BPE helps:

- Subword Representation: By merging the most frequent pairs of characters or subwords iteratively, BPE allows for the representation of less frequent words as combinations of known subwords.
- Flexibility in Vocabulary: BPE expands the vocabulary in a way that allows for greater coverage of the language without excessively increasing the number of tokens.
- Improved Contextual Understanding: When words are tokenized into subwords, the model retains more information about the input text.
- Better Generalization: With BPE, models can generalize better to unseen words or phrases.
- A Simple Byte Pair Encoding (BPE) Implementation

In this simplified approach, I wrote code to tokenize text into subword units. It reads a .txt file, splits words into tokens of 2-3 characters, and ensures that punctuation and spaces are treated as separate tokens. Here’s the key structure of the tokenizer:

- Text Normalization: Convert all characters to lowercase to ensure uniformity.
- Word Splitting: Split words longer than 3 characters into 2- or 3-character tokens.
- Punctuation Handling: Treat each punctuation and space as individual tokens.

Here’s a glance at the code:

In [12]:
import re
from collections import Counter

class BPETokenizer:
    def __init__(self, text=None):
        """Initialize the BPE Tokenizer with optional text."""
        self.text = text
        self.token_to_id = {}
        self.token_frequencies = Counter()

    def normalize_text(self):
        """Normalize the input text to lowercase."""
        if self.text is not None:
            self.text = self.text.lower()

    def tokenize(self):
        """Tokenize the text into words and punctuation."""
        return re.findall(r'\w+|[^\w\s]', self.text, re.UNICODE)

    def split_word_tokens(self, tokens):
        """Split words longer than 3 characters into 2- or 3-character tokens."""
        split_tokens = []
        
        for token in tokens:
            if token.isalpha():  # Split only alphabetic tokens
                if len(token) <= 3:
                    split_tokens.append(token)
                else:
                    # Split words into chunks of 2 or 3 characters
                    i = 0
                    while i < len(token):
                        if i + 3 <= len(token):
                            split_tokens.append(token[i:i + 3])
                            i += 3
                        else:
                            split_tokens.append(token[i:i + 2])
                            i += 2
            else:
                split_tokens.append(token)  # Keep punctuation unchanged

        return split_tokens

    def assign_token_ids(self, tokens):
        """Assign token IDs and count frequencies."""
        unique_tokens = list(set(tokens))
        self.token_to_id = {token: idx for idx, token in enumerate(unique_tokens, 1)}  # Start IDs from 1
        
        # Count the frequency of each token
        self.token_frequencies = Counter(tokens)

    def display_top_tokens(self, n=20):
        """Display the top n tokens by frequency."""
        top_tokens = self.token_frequencies.most_common(n)
        
        print(f"{'Token ID':<10} {'Token':<15} {'Frequency':<10}")
        print('-' * 35)
        for token, freq in top_tokens:
            print(f"{self.token_to_id[token]:<10} {token:<15} {freq:<10}")

    def process(self):
        """Main processing function to normalize, tokenize, and assign IDs."""
        self.normalize_text()
        tokens = self.tokenize()
        split_tokens = self.split_word_tokens(tokens)
        self.assign_token_ids(split_tokens)
        return split_tokens


def read_input_file(file_path):
    """Read input text from a .txt file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# Test the BPE tokenizer with a .txt file as input
if __name__ == "__main__":
    # Specify the path to your input file
    file_path = "data.txt"  # Replace with your actual file path
    
    # Step 1: Read the input file
    input_text = read_input_file(file_path)

    # Step 2: Create an instance of the BPETokenizer
    tokenizer = BPETokenizer(input_text)

    # Step 3: Process the text and display results
    final_tokens = tokenizer.process()
    
    print("\nTokenized Output (TokenID: Token):")
    for token in final_tokens:
        print(f"{tokenizer.token_to_id[token]}: {token}")
    
    # Step 4: Display the top 20 tokens by frequency
    print("\nTop 20 Tokens by Frequency:")
    tokenizer.display_top_tokens()



Tokenized Output (TokenID: Token):
65: in
59: a
103: qui
377: et
150: vil
2: lag
51: e
369: nes
129: tle
5: d
130: bet
357: wee
118: n
162: the
133: mou
266: nta
154: ins
229: and
162: the
169: sea
206: ,
162: the
339: peo
111: ple
8: liv
314: ed
220: pea
335: cef
404: ull
329: y
67: for
364: gen
274: era
6: tio
351: ns
77: .
162: the
40: air
95: was
363: cri
395: sp
206: ,
162: the
155: wat
29: ers
174: cle
358: ar
206: ,
229: and
182: tim
51: e
242: see
11: med
346: to
79: slo
19: w
143: dow
118: n
108: as
378: one
4: str
137: oll
314: ed
152: thr
98: oug
342: h
162: the
241: cob
46: ble
285: sto
295: ne
4: str
414: eet
338: s
77: .
197: eve
94: ry
61: mor
302: nin
343: g
206: ,
162: the
181: bak
101: ery
148: on
162: the
275: cor
107: ner
117: wou
128: ld
9: fil
165: l
162: the
40: air
22: wit
342: h
162: the
386: sce
355: nt
315: of
391: fre
376: sh
15: bre
193: ad
206: ,
306: lur
185: ing
146: bot
342: h
264: loc
310: als
229: and
30: vis
120: ito
262: rs
299: ali
244: ke
77: .
8

## Output Example
After running the tokenizer on a sample text file, here’s what the tokenized output looks like:

```
Tokenized Output (TokenID: Token):
65: in
59: a
103: qui
377: et
150: vil
2: lag
51: e
369: nes
129: tle
5: d
130: bet
357: weepy
```

Each word is split into subwords of 2–3 characters, and tokens like punctuation marks and spaces remain as individual tokens

## Token Statistics: Understanding the Distribution

After processing the text, I implemented a simple function to display the most frequent tokens. This allows us to gain insights into which subwords or punctuation marks are common in the dataset.

Here’s an example of the Top 20 tokens by frequency:

```
Top 20 Tokens by Frequency:
Token ID   Token           Frequency 
-----------------------------------
162        the             64        
206        ,               45        
77         .               38        
51         e               34        
14         t               20        
338        s               17        
59         a               16        
95         was             16        
5          d               13        
118        n               13        
346        to              13        
229        and             11        
329        y               11        
211        it              11        
65         in              10        
4          str             10        
315        of              10        
343        g               9         
150        vil             8         
2          lag             8 
```

We can see that “the” appears most frequently, which is expected for English text. Other common tokens include punctuation marks like commas and periods. By splitting words into manageable subword units, the tokenizer can handle both common and rare words efficiently.

## Byte Pair Encoding (BPE) in Action

The implementation showcased above is just a dummy example of BPE. A full-fledged BPE tokenizer would include multiple iterations of merging the most frequent pairs of characters or subwords in the vocabulary. Here’s a breakdown of how BPE works in practice:

- Initial Vocabulary: Start with individual characters.
- Pair Merging: In each iteration, merge the most frequent pair of adjacent characters or subwords.
- Token Creation: As pairs are merged, the vocabulary expands, allowing the model to represent common words as single tokens while breaking down rarer words into smaller units.

This approach allows LLMs to efficiently encode the text, minimizing the total number of tokens without sacrificing the model’s ability to generalize to new or rare words.

## Extending the Concept

Although my implementation is a simplified example, BPE is widely used in models like GPT-3 and BERT. In those models, BPE allows for highly flexible tokenization, where even unfamiliar words are tokenized into manageable subwords rather than being treated as out-of-vocabulary (OOV) words.

In real-world applications, this leads to models that can process any text efficiently, even if the exact words or phrases were not present in the training data. This feature makes BPE-based tokenizers a crucial part of modern NLP architectures.

## Conclusion

Byte Pair Encoding is essential for making large language models more efficient and flexible in handling diverse text. While this implementation is just a starting point, it provides a clear illustration of how BPE can split words into subwords, keeping the token count manageable while ensuring no word is left behind.

By experimenting with BPE, I’ve gained a deeper understanding of tokenization — a crucial step that allows modern LLMs to process massive amounts of text with ease.

Try it out! You can use the provided code to tokenize your own text and explore how different subwords and tokens are represented. Whether you’re building a full-fledged LLM or just learning about natural language processing, implementing a tokenizer is a great way to deepen your understanding.